# TP - Optimisation et Graphes

---

## Auteurs

- **Robin JUAN** - [rob1juan](https://github.com/rob1juan)
- **Maxime GUIOT** - [Grand0x](https://github.com/Grand0x)

École : [INSA Hauts-de-France](https://www.insa-hautsdefrance.fr/), 2024

## Description

Ce projet a pour objectif d'appliquer des techniques avancées d'optimisation et d'algorithmique sur des graphes. Il est divisé en deux parties principales :
* **Partie 1 :** La **résolution de problèmes d'optimisation linéaire** à l'aide de CPLEX et l'implémentation de l'**algorithme A*** pour trouver les chemins optimaux dans des graphes.


* **Partie 2 :** La résolution du **problème du voyageur de commerce (TSP)**, également à l'aide de CPLEX, pour identifier la solution optimale.

## Installation

<b><span style="color: #DC3545">ATTENTION</span></b>, pour exploiter pleinement les fonctionnalités de ce projet, veillez à utiliser un kernel (noyau) Python avec Cplex de configurer.

## Lien du Repo GitHub

Pour plus d'informations, le code source, et pour suivre le développement, [visitez le projet sur GitHub](https://github.com/).

---

## Partie 1 : Optimisation Linéaire et Algorithme de Cheminement

### 1.1 Construction du graphe

Définition d'une classe Graph :

In [126]:
class Graph:
    def __init__(self):
        self.graph = {}
        self.num_vertices = 0

    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = []
            self.num_vertices += 1

    def add_edge(self, vertex1, vertex2, cost=1):
        if vertex1 not in self.graph:
            self.add_vertex(vertex1)
        if vertex2 not in self.graph:
            self.add_vertex(vertex2)
        self.graph[vertex1].append((vertex2, cost))
        self.graph[vertex2].append((vertex1, cost))

    def display(self):
        for vertex in self.graph:
            for neighbour, cost in self.graph[vertex]:
                print(f"{vertex} --({cost})--> {neighbour}")


Définition d'une fonction pour construire des graphes :


In [127]:
import math

def build_graph_from_file(file_path):
    graph = Graph()
    with open(file_path, 'r') as file:
        n, m = map(int, file.readline().strip().split())
        grid = [file.readline().strip().split() for _ in range(n)]
    
    for row in range(n):
        for col in range(m):
            val = grid[row][col]
            if val in ['1', '2', '3']:  # Praticable ou départ/arrivée
                graph.add_vertex((row, col))
                # Ajouter des arêtes
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]:
                    nx, ny = row + dx, col + dy
                    if 0 <= nx < n and 0 <= ny < m and grid[nx][ny] in ['1', '2', '3']:
                        cost = 1 if dx == 0 or dy == 0 else math.sqrt(2)
                        graph.add_edge((row, col), (nx, ny), cost)
    return graph

Vérification de la construction du graphe avec les différents réseaux disponible dans le dossier ```/exos``` :

In [128]:
# Pour chaque réseaux dans /exos, construire le graphe et l'afficher
import os

for file in os.listdir('exos'):
    if file.endswith('.txt'):
        print(f"=== {file} ===")
        graph = build_graph_from_file(os.path.join('exos', file))
        graph.display()
        print()

=== reseau_5_10_1.txt ===
(0, 2) --(1.4142135623730951)--> (1, 3)
(0, 2) --(1.4142135623730951)--> (1, 3)
(1, 3) --(1.4142135623730951)--> (0, 2)
(1, 3) --(1.4142135623730951)--> (0, 4)
(1, 3) --(1.4142135623730951)--> (0, 2)
(1, 3) --(1.4142135623730951)--> (2, 4)
(1, 3) --(1.4142135623730951)--> (0, 4)
(1, 3) --(1.4142135623730951)--> (2, 2)
(1, 3) --(1.4142135623730951)--> (2, 2)
(1, 3) --(1.4142135623730951)--> (2, 4)
(0, 4) --(1.4142135623730951)--> (1, 3)
(0, 4) --(1.4142135623730951)--> (1, 3)
(0, 8) --(1.4142135623730951)--> (1, 9)
(0, 8) --(1.4142135623730951)--> (1, 9)
(1, 9) --(1.4142135623730951)--> (0, 8)
(1, 9) --(1)--> (2, 9)
(1, 9) --(1.4142135623730951)--> (0, 8)
(1, 9) --(1)--> (2, 9)
(2, 4) --(1.4142135623730951)--> (1, 3)
(2, 4) --(1)--> (3, 4)
(2, 4) --(1.4142135623730951)--> (1, 3)
(2, 4) --(1.4142135623730951)--> (3, 5)
(2, 4) --(1.4142135623730951)--> (3, 3)
(2, 4) --(1.4142135623730951)--> (3, 3)
(2, 4) --(1)--> (3, 4)
(2, 4) --(1.4142135623730951)--> (3, 5)
(2

### 1.2 Modélisation mathématique 

Le but de cette modélisation est de résoudre le problème du plus court chemin dans un graphe pondéré non orienté en utilisant la programmation linéaire. Nous cherchons à minimiser le coût total pour aller d'un sommet de départ à un sommet d'arrivée.

#### **Variables**

Pour chaque arête `(i, j)` du graphe, nous introduisons une variable binaire `x_ij` :
- `x_ij = 1` si l'arête est utilisée dans le chemin,
- `x_ij = 0` sinon.

#### **Fonction Objectif**

La fonction objectif est de minimiser le coût total des arêtes sélectionnées dans le chemin :

$$
\text{Minimiser} \quad Z = \sum_{(i, j) \in E} c_{ij} \cdot x_{ij}
$$

où `c_ij` est le coût de l'arête entre les sommets `i` et `j`, et `E` est l'ensemble des arêtes du graphe.

#### **Contraintes**

##### _Contraintes de Flux_

Pour chaque sommet (à l'exception du sommet de départ `s` et du sommet d'arrivée `t`), la somme des variables entrantes doit être égale à la somme des variables sortantes, ce qui garantit la continuité du chemin :

$$
\sum_{(i, v) \in E} x_{iv} - \sum_{(v, j) \in E} x_{vj} = 0 \quad \forall v \in V \setminus \{s, t\}
$$

##### _Contrainte de Départ et d'Arrivée_

Le chemin doit commencer au sommet de départ et se terminer au sommet d'arrivée :

$$
\sum_{(s, j) \in E} x_{sj} = 1, \quad \sum_{(i, t) \in E} x_{it} = 1
$$

--

Implémentation et résolution du problème linéaire avec Cplex :

In [129]:
from cplex import Cplex, SparsePair
from cplex.exceptions import CplexError

def solve_shortest_path(graph, start_vertex, end_vertex):
    prob = Cplex()
    prob.set_problem_type(Cplex.problem_type.LP)
    prob.set_log_stream(None)
    prob.set_error_stream(None)
    prob.set_warning_stream(None)
    prob.set_results_stream(None)

    edge_vars = {}
    for v in graph.graph:
        for w, cost in graph.graph[v]:
            if v < w or (w, v) not in edge_vars:  # Assurez-vous que l'arête n'est pas déjà ajoutée
                var_name = f"x{v}_{w}"
                edge_vars[(v, w)] = var_name
                edge_vars[(w, v)] = var_name  # Marquer l'arête comme ajoutée pour les deux directions
                prob.variables.add(names=[var_name], types=["B"], obj=[cost])

    prob.objective.set_sense(prob.objective.sense.minimize)

    # Création des contraintes pour garantir la connectivité
    for v in graph.graph:
        incident_edges = set()  # Utiliser un ensemble pour éviter les doublons
        for w, _ in graph.graph[v]:
            incident_edges.add(edge_vars[(v, w)])

        rhs = 2 if v == start_vertex or v == end_vertex else 1
        prob.linear_constraints.add(
            lin_expr=[SparsePair(list(incident_edges), [1.0] * len(incident_edges))],
            senses="E",
            rhs=[rhs]
        )

    prob.solve()

    # Vérifier si une solution est trouvée
    if prob.solution.get_status() == prob.solution.status.optimal:
        solution_values = prob.solution.get_values()
        path = [var for var, x in zip(prob.variables.get_names(), solution_values) if x > 0.9]
        return path
    else:
        print("Aucune solution existe.")
        return []


# Sauvegarde de la solution dans un fichier texte
def save_solution_to_file(path, input_file_name):
    output_file_name = f"sol_{input_file_name}"
    with open(output_file_name, 'w') as file:
        for step in path:
            file.write(f"{step}\n")

def extract_path(solution, edge_vars):
    path = []
    for var, value in zip(edge_vars, solution):
        if value > 0.5:
            path.append(var)
    return path

Test de la résolution du problème sur des petits réseaux :

In [130]:
import os

for file in os.listdir('exos'):
    if (file == 'exo1.txt' or file == 'exo2.txt'):
        print(f"=== {file} ===")
        # Construire le graphe à partir du fichier
        graph = build_graph_from_file(os.path.join('exos', file))
        # Assumer que start_vertex et end_vertex sont définis correctement pour chaque cas
        start_vertex = (0, 1)  # Assurez-vous que ceci est correct pour votre graphe
        end_vertex = (3, 6)    # Assurez-vous que ceci est correct pour votre graphe
        # Résoudre le chemin le plus court
        path = solve_shortest_path(graph, start_vertex, end_vertex)
        print(path)
        # Sauvegarder le résultat
        save_solution_to_file(path, file)

=== exo1.txt ===
Aucune solution existe.
[]
=== exo2.txt ===
Aucune solution existe.
[]


### 1.3 Algorithme de cheminement

In [131]:
from queue import PriorityQueue
import math

def algo_a_star(graph, start, goal, use_heuristic=True):
    """
    Implémente l'algorithme A* pour trouver le chemin le plus court entre deux sommets dans un graphe.
    
    :param graph: Une instance de la classe Graph.
    :param start: Le sommet de départ.
    :param goal: Le sommet de destination.
    :return: Le chemin le plus court sous forme de liste de sommets, du départ à la destination.
    """

    def heuristic(a, b):
        """
        Calcule l'heuristique h(x) comme la distance euclidienne entre deux points a et b.
        """
        return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) if use_heuristic else 0

    open_set = PriorityQueue()
    open_set.put((0, start))  # La queue prioritaire contient des tuples (f_score, vertex)
    came_from = {}  # Dictionnaire pour reconstruire le chemin
    
    g_score = {vertex: float('inf') for vertex in graph.graph}
    g_score[start] = 0
    
    f_score = {vertex: float('inf') for vertex in graph.graph}
    f_score[start] = heuristic(start, goal)
    
    while not open_set.empty():
        current = open_set.get()[1]
        
        if current == goal:
            # Reconstruire le chemin
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)  # Ajouter le point de départ
            return path[::-1]  # Inverser le chemin
        
        for neighbor, cost in graph.graph[current]:
            tentative_g_score = g_score[current] + cost
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                if not any(neighbor == item[1] for item in open_set.queue):
                    open_set.put((f_score[neighbor], neighbor))
    
    return []  # Retourner un chemin vide si aucun chemin n'est trouvé

In [132]:
import time

# Importer les fonctions nécessaires
from graph_builder import build_graph_from_file
from algo_a_star import algo_a_star

# Liste des noms de fichiers
files = [
    "exo1.txt",
    "exo2.txt",
    "reseau_5_10_1.txt",
    "reseau_5_10_2.txt",
    "reseau_10_10_1.txt",
    "reseau_10_10_2.txt",
    "reseau_20_20_1.txt",
    "reseau_50_50_1.txt",
]

def find_start_goal(file_path):
    start = goal = None
    with open(file_path, 'r') as file:
        n, m = map(int, file.readline().strip().split())
        for row, line in enumerate(file):
            values = line.strip().split()
            for col, value in enumerate(values):
                if value == '2':  # Départ
                    start = (row, col)
                elif value == '3':  # Arrivée
                    goal = (row, col)
    return start, goal

# Parcourir la liste des fichiers
for file in files:
    # Chemin vers votre fichier de données
    file_path = "exos/" + file

    # Construction du graphe à partir du fichier
    graph = build_graph_from_file(file_path)

    # Identifier les coordonnées de départ et d'arrivée
    start, goal = find_start_goal(file_path)

    if start and goal:
        print("-----------------")
        # Mesurer le temps d'exécution avec la heuristique euclidienne
        start_time = time.time()
        path_euclidean = algo_a_star(graph, start, goal, use_heuristic=True)
        euclidean_time = time.time() - start_time
        print(f"Chemin trouvé avec heuristique euclidienne pour {file} :", path_euclidean)
        print(f"Temps d'exécution (heuristique euclidienne): {euclidean_time:.4f} secondes")

        # Mesurer le temps d'exécution avec h(x) = 0 (la fonction heuristic renvoie 0)
        start_time = time.time()
        path_no_heuristic = algo_a_star(graph, start, goal, use_heuristic=False)
        no_heuristic_time = time.time() - start_time
        print(f"Chemin trouvé avec h(x) = 0 pour {file} :", path_no_heuristic)
        print(f"Temps d'exécution (h(x) = 0): {no_heuristic_time:.4f} secondes")

        # Comparer les longueurs des chemins
        euclidean_length = len(path_euclidean)
        no_heuristic_length = len(path_no_heuristic)
        print(f"Longueur du chemin (heuristique euclidienne): {euclidean_length}")
        print(f"Longueur du chemin (h(x) = 0): {no_heuristic_length}")

        # Analyser et commenter les résultats
        if euclidean_length == no_heuristic_length:
            print("Les deux chemins ont la même longueur.")
        else:
            print("Les chemins ont des longueurs différentes.")
            
        if euclidean_time < no_heuristic_time:
            print("L'heuristique euclidienne a été plus rapide.")
        elif euclidean_time > no_heuristic_time:
            print("L'heuristique avec h(x) = 0 a été plus rapide.")
        else:
            print("Les deux heuristiques ont eu des temps d'exécution similaires.")

        # Sauvegarder le chemin dans un fichier
        with open("sol/sol_" + file, "w") as f:
            for node in path_euclidean:
                f.write(f"{node}\n")
    else:
        print(f"Impossible de trouver le départ ou l'arrivée pour {file}.")


-----------------
Chemin trouvé avec heuristique euclidienne pour exo1.txt : [(0, 1), (1, 2), (1, 3), (1, 4), (2, 5), (3, 6)]
Temps d'exécution (heuristique euclidienne): 0.0000 secondes
Chemin trouvé avec h(x) = 0 pour exo1.txt : [(0, 1), (0, 2), (0, 3), (1, 4), (2, 5), (3, 6)]
Temps d'exécution (h(x) = 0): 0.0000 secondes
Longueur du chemin (heuristique euclidienne): 6
Longueur du chemin (h(x) = 0): 6
Les deux chemins ont la même longueur.
Les deux heuristiques ont eu des temps d'exécution similaires.
-----------------
Chemin trouvé avec heuristique euclidienne pour exo2.txt : []
Temps d'exécution (heuristique euclidienne): 0.0000 secondes
Chemin trouvé avec h(x) = 0 pour exo2.txt : []
Temps d'exécution (h(x) = 0): 0.0000 secondes
Longueur du chemin (heuristique euclidienne): 0
Longueur du chemin (h(x) = 0): 0
Les deux chemins ont la même longueur.
Les deux heuristiques ont eu des temps d'exécution similaires.
-----------------
Chemin trouvé avec heuristique euclidienne pour reseau_5

---

## Partie 2 : Problème du Voyageur de Commerce (TSP)

Adaptation de la fonction build_graph pour TSP :

In [133]:
import random
from cplex import Cplex, SparsePair
from cplex.exceptions import CplexError

def load_graph_from_file(file_path):
    """ Charge les données d'un graphe depuis un fichier. """
    with open(file_path, 'r') as file:
        num_vertices, num_edges = map(int, file.readline().split())
        edges = []
        for _ in range(num_edges):
            u, v, cost = map(int, file.readline().split())
            edges.append((u, v, cost))
    return num_vertices, edges


--

### Représentation du Problème Linéaire du Voyageur de Commerce (TSP)

#### **Variables**

Pour chaque paire de sommets \( i, j \) dans le graphe, nous introduisons une variable binaire \( x_{ij} \) :
- \( x_{ij} = 1 \) si l'arête \( (i, j) \) est utilisée dans le parcours,
- \( x_{ij} = 0 \) sinon.

#### **Fonction Objectif**

La fonction objectif est de minimiser le coût total du parcours, c'est-à-dire la somme des coûts des arêtes utilisées dans le parcours :

$$
\text{Minimiser} \quad Z = \sum_{(i, j) \in E} c_{ij} \cdot x_{ij}
$$

où \( c_{ij} \) représente le coût de l'arête entre les sommets \( i \) et \( j \), et \( E \) est l'ensemble des arêtes du graphe.

#### **Contraintes**

##### _Contraintes de Visite_

Chaque sommet doit être visité exactement une fois. Cela signifie que pour chaque sommet, exactement deux arêtes doivent être utilisées (une entrante et une sortante) :

$$
\sum_{(i, v) \in E} x_{iv} + \sum_{(v, j) \in E} x_{vj} = 2 \quad \forall v \in V
$$

##### _Contraintes d'Élimination des Sous-Tours_

Pour éviter les sous-tours, c'est-à-dire les tours qui ne visitent pas tous les sommets, nous utilisons les inégalités subtour :

Pour chaque sous-ensemble \( S \) de sommets avec \( S \subset V \) et \( |S| > 1 \) :

$$
\sum_{i \in S, j \in S, i \neq j} x_{ij} \leq |S| - 1
$$

Ces contraintes assurent que si un sous-ensemble de sommets est connecté entre eux, ils ne forment pas un circuit fermé sans inclure au moins un sommet externe à ce sous-ensemble, ce qui force le tour à être global et à inclure tous les sommets.

--

Résolution avec Cplex :

In [135]:
def solve_tsp(num_vertices, edges):
    """ Résout le problème du voyageur de commerce (TSP) en utilisant CPLEX. """
    prob = Cplex()
    prob.set_problem_type(Cplex.problem_type.LP)
    prob.set_log_stream(None)
    prob.set_error_stream(None)
    prob.set_warning_stream(None)
    prob.set_results_stream(None)
    
    # Préparation des variables et de la fonction objectif
    edge_vars = {}
    for i, j, cost in edges:
        var_name = f"x{min(i, j)}_{max(i, j)}"
        edge_vars[(min(i, j), max(i, j))] = var_name
        prob.variables.add(names=[var_name], types=["B"], obj=[cost])

    # Ajouter des contraintes pour assurer la visite de chaque sommet
    for v in range(num_vertices):
        incident_edges = [edge_vars[(min(v, w), max(v, w))] for w in range(num_vertices) if v != w and (min(v, w), max(v, w)) in edge_vars]
        prob.linear_constraints.add(
            lin_expr=[SparsePair(incident_edges, [1] * len(incident_edges))],
            senses="E",
            rhs=[2]
        )
    
    try:
        prob.solve()
        solution_values = prob.solution.get_values()
        selected_edges = [e for e, x in zip(edge_vars.values(), solution_values) if x > 0.5]
        return prob, selected_edges
    except CplexError as e:
        return prob, []


Fonction pour générer des graphes :

In [136]:
def generate_random_graph(num_vertices, p, cost_range):
    """ Génère un graphe aléatoire en utilisant le modèle Erdös-Rényi. """
    edges = []
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            if random.random() < p:
                cost = random.randint(*cost_range)
                edges.append((i, j, cost))
    return num_vertices, edges


Test de l'implémentation :

In [137]:
def save_graph_to_file(num_vertices, edges, file_path):
    """ Sauvegarde le graphe généré dans un fichier. """
    with open(file_path, 'w') as file:
        file.write(f"{num_vertices} {len(edges)}\n")
        for u, v, cost in edges:
            file.write(f"{u} {v} {cost}\n")

def main():
    """ Fonction principale pour exécuter et tester le solveur TSP. """
    # Génération d'un graphe aléatoire
    num_vertices, edges = generate_random_graph(10, 0.5, (10, 50))
    # Sauvegarde du graphe généré dans un fichier
    save_graph_to_file(num_vertices, edges, "random_graph.txt")
    
    # Chargement et résolution du graphe sauvegardé
    num_vertices_loaded, edges_loaded = load_graph_from_file("random_graph.txt")
    
    # Affichage du graphe
    print("Graphe chargé :")
    for u, v, cost in edges:
        print(f"({u}, {v}) coût: {cost}")
    
    # Résoudre le problème TSP
    prob, selected_edges = solve_tsp(num_vertices, edges)
    if selected_edges:
        print("\nStatut de la solution : Solution optimale trouvée")
        print("Valeur de l'objectif : ", prob.solution.get_objective_value())
        print("Arêtes sélectionnées : ", selected_edges)
    else:
        print("\nPas de solution optimale trouvée.")

if __name__ == "__main__":
    main()


Graphe chargé :
(0, 3) coût: 15
(0, 6) coût: 24
(0, 8) coût: 25
(0, 9) coût: 35
(1, 2) coût: 11
(1, 3) coût: 21
(1, 4) coût: 12
(1, 6) coût: 50
(1, 8) coût: 43
(1, 9) coût: 43
(2, 5) coût: 28
(2, 7) coût: 45
(2, 9) coût: 42
(3, 5) coût: 15
(4, 6) coût: 16
(4, 8) coût: 38
(5, 8) coût: 27
(6, 7) coût: 48
(6, 9) coût: 50
(7, 9) coût: 22
(8, 9) coût: 27

Statut de la solution : Solution optimale trouvée
Valeur de l'objectif :  214.0
Arêtes sélectionnées :  ['x0_3', 'x0_6', 'x1_2', 'x1_4', 'x2_7', 'x3_5', 'x4_6', 'x5_8', 'x7_9', 'x8_9']
